In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss, mean_squared_error, mean_absolute_error
import gezi

In [2]:
import tensorflow as tf
import torch 
tf.enable_eager_execution()

In [3]:
data = tf.random_normal([2, 8, 3])

In [4]:
data

<tf.Tensor: id=5, shape=(2, 8, 3), dtype=float32, numpy=
array([[[ 0.63228077,  0.86531305, -0.3053677 ],
        [-0.6178    ,  1.2101597 , -0.26687023],
        [-0.7250177 , -0.86864716, -1.5621979 ],
        [-1.2312133 ,  1.1414009 , -0.84329027],
        [ 0.5682497 ,  0.95293283, -1.3176906 ],
        [ 1.3617997 , -0.00987865, -1.4242511 ],
        [ 0.3111133 ,  0.8858375 , -0.29353368],
        [-0.22367527, -0.53028625, -0.05714251]],

       [[ 0.96788794, -0.67219263, -0.22908396],
        [ 1.3660717 ,  1.0273248 , -0.9666571 ],
        [ 1.4951528 ,  0.65344167,  1.7204219 ],
        [ 0.844709  , -0.07005837, -0.00774324],
        [-1.2686813 ,  0.89502805, -0.18682602],
        [ 0.60661644, -1.1651388 , -1.7967056 ],
        [-0.72723204,  0.14008093, -1.8052537 ],
        [-0.78192294, -1.6477147 , -0.41716218]]], dtype=float32)>

In [5]:
seg_ids = tf.constant([[1, 2, 5, 3, 1, 3, 5, 5], [2, 2, 3, 3, 1, 1, 4, 4]])

In [6]:
seg_ids

<tf.Tensor: id=7, shape=(2, 8), dtype=int32, numpy=
array([[1, 2, 5, 3, 1, 3, 5, 5],
       [2, 2, 3, 3, 1, 1, 4, 4]], dtype=int32)>

In [12]:
seg_ids_ = tf.tile(tf.expand_dims(seg_ids, -1), [1, 1, melt.get_shape(data, -1)])

In [13]:
seg_ids_

<tf.Tensor: id=35, shape=(2, 8, 3), dtype=int32, numpy=
array([[[1, 1, 1],
        [2, 2, 2],
        [5, 5, 5],
        [3, 3, 3],
        [1, 1, 1],
        [3, 3, 3],
        [5, 5, 5],
        [5, 5, 5]],

       [[2, 2, 2],
        [2, 2, 2],
        [3, 3, 3],
        [3, 3, 3],
        [1, 1, 1],
        [1, 1, 1],
        [4, 4, 4],
        [4, 4, 4]]], dtype=int32)>

In [19]:
list(melt.get_shape(data))

[Dimension(2), Dimension(8), Dimension(3)]

In [21]:
ref = tf.zeros([melt.get_shape(data, 0), 6] + [melt.get_shape(data, -1)])

In [22]:
ref

<tf.Tensor: id=60, shape=(2, 6, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)>

In [36]:
tf.tensor_scatter_add(tf.transpose(ref, [2, 0, 1]), tf.transpose(seg_ids_, [2, 0, 1]), tf.transpose(data, [2, 0, 1]))

InvalidArgumentError: Inner dimensions of output shape must match inner dimensions of updates shape. Output: [3,2,6] updates: [3,2,8] [Op:TensorScatterAdd]

In [35]:
tf.tensor_scatter_add(ref, seg_ids_, data)

InvalidArgumentError: Inner dimensions of output shape must match inner dimensions of updates shape. Output: [2,6,3] updates: [2,8,3] [Op:TensorScatterAdd]

In [7]:
import melt

In [8]:
melt.unsorted_segment_embs(data, seg_ids, 6)

<tf.Tensor: id=26, shape=(2, 6, 3), dtype=float32, numpy=
array([[[ 0.        ,  0.        ,  0.        ],
        [ 1.2005305 ,  1.8182459 , -1.6230583 ],
        [-0.6178    ,  1.2101597 , -0.26687023],
        [ 0.13058639,  1.1315223 , -2.2675414 ],
        [ 0.        ,  0.        ,  0.        ],
        [-0.6375797 , -0.5130959 , -1.9128741 ]],

       [[ 0.        ,  0.        ,  0.        ],
        [-0.66206485, -0.2701108 , -1.9835316 ],
        [ 2.3339596 ,  0.35513216, -1.195741  ],
        [ 2.3398619 ,  0.5833833 ,  1.7126787 ],
        [-1.509155  , -1.5076338 , -2.222416  ],
        [ 0.        ,  0.        ,  0.        ]]], dtype=float32)>

In [9]:
import lele

In [30]:
def unsorted_segment_sum_(data, segment_ids, num_segments):
    """
    Computes the sum along segments of a tensor. Analogous to tf.unsorted_segment_sum.

    :param data: A tensor whose segments are to be summed.
    :param segment_ids: The segment indices tensor.
    :param num_segments: The number of segments.
    :return: A tensor of same data type as the data argument.
    """
    segment_ids = torch.repeat_interleave(segment_ids.unsqueeze(-1), repeats=data.shape[-1], dim=-1)

    shape = [data.shape[0], num_segments] + list(data.shape[2:])
    print(shape, seg_ids.shape, data.shape)
    tensor = torch.zeros(*shape).scatter_add(1, segment_ids, data)
    # tensor = torch.zeros(*shape).cuda().scatter_add(1, segment_ids, data)
    return tensor

In [11]:
lele.unsorted_segment_sum(torch.as_tensor(data.numpy()), torch.as_tensor(seg_ids.numpy()).long(), 6)

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 1.2005,  1.8182, -1.6231],
         [-0.6178,  1.2102, -0.2669],
         [ 0.1306,  1.1315, -2.2675],
         [ 0.0000,  0.0000,  0.0000],
         [-0.6376, -0.5131, -1.9129]],

        [[ 0.0000,  0.0000,  0.0000],
         [-0.6621, -0.2701, -1.9835],
         [ 2.3340,  0.3551, -1.1957],
         [ 2.3399,  0.5834,  1.7127],
         [-1.5092, -1.5076, -2.2224],
         [ 0.0000,  0.0000,  0.0000]]])

In [31]:
unsorted_segment_sum_(torch.as_tensor(data.numpy()), torch.as_tensor(seg_ids.numpy()).long(), 6)

[2, 6, 3] (2, 8) torch.Size([2, 8, 3])


tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 1.2005,  1.8182, -1.6231],
         [-0.6178,  1.2102, -0.2669],
         [ 0.1306,  1.1315, -2.2675],
         [ 0.0000,  0.0000,  0.0000],
         [-0.6376, -0.5131, -1.9129]],

        [[ 0.0000,  0.0000,  0.0000],
         [-0.6621, -0.2701, -1.9835],
         [ 2.3340,  0.3551, -1.1957],
         [ 2.3399,  0.5834,  1.7127],
         [-1.5092, -1.5076, -2.2224],
         [ 0.0000,  0.0000,  0.0000]]])

In [ ]:
indices = tf.constant([[0], [2]])
updates = tf.constant([[[5, 5, 5, 5], [6, 6, 6, 6],
                        [7, 7, 7, 7], [8, 8, 8, 8]],
                       [[5, 5, 5, 5], [6, 6, 6, 6],
                        [7, 7, 7, 7], [8, 8, 8, 8]]])
tensor = tf.ones([4, 4, 4],dtype=tf.int32)
updated = tf.tensor_scatter_nd_add(tensor, indices, updates)
print(updated)

In [ ]:
x = tf.constant([[1., 2.], [3., 4]])

In [ ]:
x

In [ ]:
tf.expand_dims(x, -1)

In [ ]:
tf.tile(tf.expand_dims(x, -1), [1, 1, 3])

In [ ]:
x_ = tf.constant([[1., 2.], [3., 4]])

In [ ]:
y = torch.as_tensor(x_.numpy())

In [ ]:
y

In [ ]:
torch.repeat_interleave(y.unsqueeze(-1), 3, dim=-1)[:,:, 1]

In [16]:
torch.repeat_interleave(torch.as_tensor(seg_ids.numpy()).unsqueeze(-1), 3, dim=-1)

tensor([[[1, 1, 1],
         [2, 2, 2],
         [5, 5, 5],
         [3, 3, 3],
         [1, 1, 1],
         [3, 3, 3],
         [5, 5, 5],
         [5, 5, 5]],

        [[2, 2, 2],
         [2, 2, 2],
         [3, 3, 3],
         [3, 3, 3],
         [1, 1, 1],
         [1, 1, 1],
         [4, 4, 4],
         [4, 4, 4]]], dtype=torch.int32)

In [ ]:
indices = tf.constant([[4], [3], [1], [7]])
updates = tf.constant([9, 10, 11, 12])
tensor = tf.ones([8], dtype=tf.int32)
updated = tf.tensor_scatter_nd_add(tensor, indices, updates)

In [ ]:
tensor

In [ ]:
a, b, c = 3, 4, 5
memory = tf.ones([a, b, c])
indices = tf.constant([[2], [0], [3]])
updates = 10 * tf.reshape(tf.range(a * c, dtype=memory.dtype), [a, c])
print(updates.numpy())
# [[  0.  10.  20.  30.  40.]
#  [ 50.  60.  70.  80.  90.]
#  [100. 110. 120. 130. 140.]]

# Make indices for first dimension
ind_a = tf.range(tf.shape(indices, out_type=indices.dtype)[0])
# Make full indices
indices_2 = tf.concat([tf.expand_dims(ind_a, 1), indices], axis=1)
# Scatter add
out = tf.tensor_scatter_nd_add(memory, indices_2, updates)
print(out.numpy())

In [ ]:
d = pd.read_csv('/home/gezi/tmp/rank/exps/video/v25/dlrm-active/infos/2020060723/valid.csv')

In [ ]:
d

In [ ]:
d2 = pd.read_csv('/home/gezi/tmp/rank/exps/video/v25/8/2020060723/valid.csv')

In [ ]:
d2 = d2[d2.product_data=='sgsapp']

In [ ]:
d2 = d2[d2.abtestid.isin([4,5,6])]

In [ ]:
d2 = d2.sort_values(['mid', 'impression_time', 'position']).reset_index()

In [ ]:
d2.mid

In [ ]:
d = d.sort_values(['mid', 'impression_time', 'position']).reset_index()

In [ ]:
d.mid

In [ ]:
del d['article_page_time']

In [ ]:
# d = d.merge(d2[['docid', 'impression_time', 'article_page_time']], on=['docid', 'impression_time'], how='left')

In [ ]:
d.columns

In [ ]:
d[['mid', 'docid', 'position', 'impression_time']]

In [ ]:
d2[['mid', 'docid', 'position', 'impression_time']]

In [ ]:
d = pd.concat([d, d2.article_page_time], axis=1)

In [ ]:
d

In [ ]:
d[['mid', 'docid', 'position', 'impression_time', 'article_page_time']]

In [ ]:
d2[['mid', 'docid', 'position', 'impression_time', 'article_page_time']]

In [ ]:
d['page_time'] = ((d.impression_time - d.article_page_time) / 3600).astype(int)

In [ ]:
import time

In [ ]:
d2['page_time'] = ((d2.impression_time - d2.article_page_time) / 3600).astype(int)

In [ ]:
time.localtime(d.impression_time.values[0])

In [ ]:
d.page_time

In [ ]:
d2.page_time

In [ ]:
roc_auc_score(d.duration != 0, d.pred_click)

In [ ]:
roc_auc_score(d2.duration != 0, d2.pred_click)

In [ ]:
d_1 = d[d.page_time>720]

In [ ]:
d_1

In [ ]:
d

In [ ]:
roc_auc_score(d_1.duration != 0, d_1.pred_click)

In [ ]:
d2_1 =  d2[d2.page_time>720]

In [ ]:
roc_auc_score(d2_1.duration != 0, d2_1.pred_click)

In [ ]:
d_2 = d[d.page_time<240]

In [ ]:
d2_2 = d2[d2.page_time>240]

In [ ]:
roc_auc_score(d_2.duration != 0, d_2.pred_click)

In [ ]:
roc_auc_score(d2_2.duration != 0, d2_2.pred_click)

In [ ]:
d2_2

In [ ]:
d.page_time.describe()

In [ ]:
gezi.metrics.group_auc(d.duration != 0, d.pred_click, d.mid + '\t' + d.impression_time.astype(str))

In [ ]:
gezi.metrics.group_auc(d2.duration != 0, d2.pred_click, d2.mid + '\t' + d2.impression_time.astype(str))

In [ ]:
d_3 = d[d.page_time > 720]

In [ ]:
d_3

In [ ]:
d2_3 = d2[d2.page_time > 720]

In [ ]:
def group_auc(d):
  return gezi.metrics.group_auc((d.duration != 0).values, d.pred_click.values, (d.mid + '\t' + d.impression_time.astype(str)).values)

In [ ]:
group_auc(d_3)

In [ ]:
group_auc(d2_3)

In [ ]:
d['num_histories'] = d.num_tw_histories + d.num_vd_histories

In [ ]:
d

In [ ]:
d2

In [ ]:
d2 = pd.concat([d2, d.num_histories], axis=1)

In [ ]:
d2

In [ ]:
d_3 = d[d.num_histories>=100]

In [ ]:
d_3

In [ ]:
d2_3 = d2[d2.num_histories>=100]

In [ ]:
d2_3

In [ ]:
group_auc(d_3)

In [ ]:
group_auc(d2_3)